In [1]:
import subprocess
from Bio import Entrez
from Bio import SeqIO
import pandas as pd
Entrez.email = "o.william.white@gmail.com"

In [2]:
# input parameters
reference_accession = "NC_000932.1"
query_accession = "NC_000932.1"
query_circular = True

In [3]:
## download genbank of reference data and write annotated genes to fasta

# efetch accession
handle = Entrez.efetch(db="nucleotide", id=reference_accession, rettype="gb", retmode="text")
reference_record = SeqIO.read(handle, "gb")
handle.close()

# list and dictionary to store annotations and counts respectively
list_coding, list_non_coding, dict_genes, dict_counts = [], [], {}, {}

# loop through sequence feature and append to list
for feature in reference_record.features:
    # cds features
    if (feature.type in ["CDS", "tRNA", "rRNA"]) and ("gene" in feature.qualifiers.keys()):
        # get annotation info
        gene_name = str(feature.qualifiers["gene"][0])
        gene_seq = feature.extract(reference_record)
        gene_seq.description = ""
        if dict_genes.get(gene_name) == None:
            dict_genes[gene_name] = str(gene_seq.seq)
            dict_counts[gene_name] = 1
        elif dict_genes[gene_name] == str(gene_seq.seq): 
            print(f"Ignoring duplicate annotation for {gene_name}")
            continue
        elif dict_genes[gene_name] != str(gene_seq.seq):
            dict_counts[gene_name] += 1
            print(f"Note {dict_counts[gene_name]} sequences have been extracted under the name {gene_name}")
        if feature.type == "CDS": 
            gene_seq.id = f"cds-blatx_{gene_name}_{dict_counts[gene_name]};gbkey={feature.type};gene={gene_name}"
            list_coding.append(gene_seq)
        elif (feature.type == "tRNA") or (feature.type == "rRNA"):
            gene_seq.id = f"cds-blatn_{gene_name}_{dict_counts[gene_name]};gbkey={feature.type};gene={gene_name}"
            list_non_coding.append(gene_seq)

# write reference fasta
with open("reference_coding.fasta", "w") as coding_fas:
    SeqIO.write(list_coding, coding_fas, "fasta")

with open("reference_non_coding.fasta", "w") as non_coding_fas:
    SeqIO.write(list_non_coding, non_coding_fas, "fasta")

Note 2 sequences have been extracted under the name trnS
Note 2 sequences have been extracted under the name trnG
Note 3 sequences have been extracted under the name trnS
Ignoring duplicate annotation for rps12
Note 2 sequences have been extracted under the name trnL
Note 2 sequences have been extracted under the name trnV
Note 2 sequences have been extracted under the name trnI
Note 2 sequences have been extracted under the name trnR
Note 3 sequences have been extracted under the name trnL
Note 2 sequences have been extracted under the name ycf1
Ignoring duplicate annotation for trnN
Note 3 sequences have been extracted under the name trnR
Ignoring duplicate annotation for trnA
Note 3 sequences have been extracted under the name trnI
Ignoring duplicate annotation for rps7
Ignoring duplicate annotation for ndhB
Note 4 sequences have been extracted under the name trnL
Ignoring duplicate annotation for ycf2
Ignoring duplicate annotation for trnI
Ignoring duplicate annotation for rpl23
Ig

In [4]:
## download query accession and write to fasta

# efetch accession
handle = Entrez.efetch(db="nucleotide", id=query_accession, rettype="fasta", retmode="text")
query_record = SeqIO.read(handle, "fasta")
handle.close()

# if the query is circular, append the first 10Kb to the end of the sequence query
if query_circular:
    print("Treating query as circular")
    query_record.seq = query_record.seq + query_record.seq[0:10000]

# write reference fasta
with open("query.fasta", "w") as que:
    SeqIO.write(query_record, que, "fasta")

Treating query as circular


In [5]:
# run blat
subprocess.run(["blat", "reference_coding.fasta",      "query.fasta", "-t=dnax", "-q=dnax", "-out=blast8", "output_coding.txt"])
subprocess.run(["blat", "reference_non_coding.fasta",  "query.fasta", "-t=dna",  "-q=dna",  "-out=blast8", "output_non_coding.txt"])

Loaded 69111 letters in 79 sequences
Blatx 79 sequences in database, 1 files in query
Loaded 6829 letters in 35 sequences
Searched 164478 bases in 1 sequences


CompletedProcess(args=['blat', 'reference_non_coding.fasta', 'query.fasta', '-t=dna', '-q=dna', '-out=blast8', 'output_non_coding.txt'], returncode=0)

In [6]:
# read blat output into pandas dataframes
blast8_columns = ["qseqid", "sseqid", "pident", "length", "mismatch", "gapopen", "qstart", "qend", "sstart", "send", "evalue", "bitscore"]
blat_coding     = pd.read_csv("output_coding.txt",     sep="\t", names=blast8_columns)
blat_non_coding = pd.read_csv("output_non_coding.txt", sep="\t", names=blast8_columns)

In [7]:
# check 
blat_coding

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
0,NC_000932.1,cds-blatx_ycf2_1;gbkey=CDS;gene=ycf2,100.00,6885,0,0,86474,93358,1,6885,0.000000e+00,13318.0
1,NC_000932.1,cds-blatx_ycf2_1;gbkey=CDS;gene=ycf2,100.00,6885,0,0,145291,152175,6885,1,0.000000e+00,13318.0
2,NC_000932.1,cds-blatx_ycf2_1;gbkey=CDS;gene=ycf2,84.54,97,12,1,89060,89156,2611,2704,1.600000e-31,134.0
3,NC_000932.1,cds-blatx_ycf2_1;gbkey=CDS;gene=ycf2,84.54,97,12,1,149493,149589,2704,2611,1.600000e-31,134.0
4,NC_000932.1,cds-blatx_ycf2_1;gbkey=CDS;gene=ycf2,86.81,91,9,1,89083,89170,2586,2676,5.400000e-31,132.0
...,...,...,...,...,...,...,...,...,...,...,...,...
242,NC_000932.1,cds-blatx_psbI_1;gbkey=CDS;gene=psbI,100.00,111,0,0,162061,162171,1,111,1.100000e-55,214.0
243,NC_000932.1,cds-blatx_psbM_1;gbkey=CDS;gene=psbM,100.00,105,0,0,28707,28811,105,1,5.400000e-52,202.0
244,NC_000932.1,cds-blatx_psbT_1;gbkey=CDS;gene=psbT,100.00,102,0,0,74082,74183,1,102,3.400000e-50,196.0
245,NC_000932.1,cds-blatx_petL_1;gbkey=CDS;gene=petL,100.00,96,0,0,65712,65807,1,96,7.000000e-47,185.0


In [8]:
blat_non_coding

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
0,NC_000932.1,cds-blatn_rrn23S_1;gbkey=rRNA;gene=rrn23S,100.0,2810,0,0,104691,107500,1,2810,0.000000e+00,5528.0
1,NC_000932.1,cds-blatn_rrn23S_1;gbkey=rRNA;gene=rrn23S,100.0,2810,0,0,131149,133958,2810,1,0.000000e+00,5528.0
2,NC_000932.1,cds-blatn_rrn16S_1;gbkey=rRNA;gene=rrn16S,100.0,1491,0,0,101012,102502,1,1491,0.000000e+00,2937.0
3,NC_000932.1,cds-blatn_rrn16S_1;gbkey=rRNA;gene=rrn16S,100.0,1491,0,0,136147,137637,1491,1,0.000000e+00,2937.0
4,NC_000932.1,cds-blatn_rrn4.5S_1;gbkey=rRNA;gene=rrn4.5S,100.0,103,0,0,107599,107701,1,103,5.600000e-52,202.0
...,...,...,...,...,...,...,...,...,...,...,...,...
71,NC_000932.1,cds-blatn_trnI_2;gbkey=tRNA;gene=trnI,100.0,35,0,0,135048,135082,72,38,4.500000e-12,69.0
72,NC_000932.1,cds-blatn_trnK_1;gbkey=tRNA;gene=trnK,100.0,37,0,0,4311,4347,37,1,5.700000e-13,72.0
73,NC_000932.1,cds-blatn_trnK_1;gbkey=tRNA;gene=trnK,100.0,37,0,0,158789,158825,37,1,5.700000e-13,72.0
74,NC_000932.1,cds-blatn_trnK_1;gbkey=tRNA;gene=trnK,100.0,35,0,0,1717,1751,72,38,3.500000e-12,70.0


In [9]:
# select top hits based on bitscores
blat_coding_top     = blat_coding.loc[blat_coding.groupby('sseqid')['bitscore'].idxmax()]
blat_non_coding_top = blat_non_coding.loc[blat_non_coding.groupby('sseqid')['bitscore'].idxmax()]

In [10]:
# concat blat outputs
dat = pd.concat([blat_coding_top, blat_non_coding_top], axis=0)
dat = dat.sort_values(by="qstart").reset_index()
dat

,index,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
0,37,NC_000932.1,cds-blatn_trnH_1;gbkey=tRNA;gene=trnH,100.00,73,0,0,4,76,73,1,2.100000e-34,143.0
1,36,NC_000932.1,cds-blatx_psbA_1;gbkey=CDS;gene=psbA,100.00,1062,0,0,383,1444,1062,1,0.000000e+00,2062.0
2,27,NC_000932.1,cds-blatx_matK_1;gbkey=CDS;gene=matK,100.00,1515,0,0,2056,3570,1515,1,0.000000e+00,2914.0
3,72,NC_000932.1,cds-blatn_trnK_1;gbkey=tRNA;gene=trnK,100.00,37,0,0,4311,4347,37,1,5.700000e-13,72.0
4,100,NC_000932.1,cds-blatx_rps16_1;gbkey=CDS;gene=rps16,100.00,204,0,0,5084,5287,240,37,5.600000e-110,394.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,64,NC_000932.1,cds-blatx_rpl2_1;gbkey=CDS;gene=rpl2,99.55,440,2,0,153873,154312,386,825,5.300000e-248,853.0
110,106,NC_000932.1,cds-blatx_psbK_1;gbkey=CDS;gene=psbK,100.00,186,0,0,161495,161680,1,186,2.600000e-99,359.0
111,117,NC_000932.1,cds-blatx_psbI_1;gbkey=CDS;gene=psbI,100.00,111,0,0,162061,162171,1,111,1.100000e-55,214.0
112,9,NC_000932.1,cds-blatn_trnS_1;gbkey=tRNA;gene=trnS,100.00,88,0,0,162263,162350,1,88,4.700000e-43,172.0


In [11]:
dat [ dat["sseqid"] == "cds-blatx_matK_1;gbkey=CDS;gene=matK"]


,index,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
2,27,NC_000932.1,cds-blatx_matK_1;gbkey=CDS;gene=matK,100.0,1515,0,0,2056,3570,1515,1,0.0,2914.0


In [12]:
dat [ dat["sseqid"] == "cds-blatn_trnI_1;gbkey=tRNA;gene=trnI"]

,index,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
107,34,NC_000932.1,cds-blatn_trnI_1;gbkey=tRNA;gene=trnI,100.0,74,0,0,152264,152337,1,74,1.100000e-34,144.0


In [13]:
reference_record.features

[SeqFeature(FeatureLocation(ExactPosition(0), ExactPosition(154478), strand=1), type='source'),
 SeqFeature(CompoundLocation([FeatureLocation(ExactPosition(69610), ExactPosition(69724), strand=-1), FeatureLocation(ExactPosition(97998), ExactPosition(98793), strand=-1)], 'join'), type='gene', location_operator='join'),
 SeqFeature(CompoundLocation([FeatureLocation(ExactPosition(69610), ExactPosition(69724), strand=-1), FeatureLocation(ExactPosition(98561), ExactPosition(98793), strand=-1), FeatureLocation(ExactPosition(97998), ExactPosition(98024), strand=-1)], 'join'), type='CDS', location_operator='join'),
 SeqFeature(FeatureLocation(ExactPosition(3), ExactPosition(76), strand=-1), type='gene'),
 SeqFeature(FeatureLocation(ExactPosition(3), ExactPosition(76), strand=-1), type='tRNA'),
 SeqFeature(FeatureLocation(ExactPosition(382), ExactPosition(1444), strand=-1), type='gene'),
 SeqFeature(FeatureLocation(ExactPosition(382), ExactPosition(1444), strand=-1), type='CDS'),
 SeqFeature(Fe

In [15]:
dat [ dat["sseqid"] == "cds-blatx_matK_1;gbkey=CDS;gene=matK"]

,index,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
2,27,NC_000932.1,cds-blatx_matK_1;gbkey=CDS;gene=matK,100.0,1515,0,0,2056,3570,1515,1,0.0,2914.0


In [34]:
reference_record.features[6]

SeqFeature(FeatureLocation(ExactPosition(382), ExactPosition(1444), strand=-1), type='CDS')

In [ ]:
from Bio.SeqFeature import SeqFeature, FeatureLocation

feature = SeqFeature(
    location=FeatureLocation(2056, 3570, strand=-1), 
    type="CDS",
    id="matK")
feature.type = "CDS"

query_record.features.append(feature)

In [20]:
query_record.features

[SeqFeature(FeatureLocation(ExactPosition(2056), ExactPosition(3570), strand=-1), type='CDS', id='matK')]

In [26]:
# remember to remove 10kb 
query_record.annotations["molecule_type"] = "DNA"
if query_circular: 
    query_record.annotations["topology"] = "circular"
else: 
    query_record.annotations["topology"] = "linear"

In [27]:
from Bio import SeqIO

with open("example.gb", "w") as output_handle:
    SeqIO.write(query_record, output_handle, "gb")

In [38]:
# read in query
list(SeqIO.parse("query.fasta", "fasta"))[0]

SeqRecord(seq=Seq('ATGGGCGAACGACGGGAATTGAACCCGCGATGGTGAATTCACAATCCACTGCCT...GCT'), id='NC_000932.1', name='NC_000932.1', description='NC_000932.1 Arabidopsis thaliana chloroplast, complete genome', dbxrefs=[])